# PDFドキュメントの読込

# 準備

In [1]:
# 必要なモジュールをインポート
import os
from dotenv import load_dotenv

# 環境変数の読み込み
load_dotenv()
os.environ['OPENAI_API_KEY']  = os.environ['API_KEY']

# Indexの構築

In [2]:
from pathlib import Path
from llama_index.readers.file import PDFReader

# PDFドキュメントからテキスト情報を読込
loader = PDFReader()
documents = loader.load_data(file=Path('./data2/001615363.pdf'))

In [3]:
from llama_index.core.text_splitter import SentenceSplitter
import tiktoken

# NodeParserの作成
node_parser = SentenceSplitter(
    separator="。",
    chunk_size=1024,
    chunk_overlap=20,
    tokenizer=tiktoken.encoding_for_model("gpt-3.5-turbo-1106").encode)

In [6]:
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex
from llama_index.core import Settings

# 言語モデルの指定
llm = OpenAI(model='gpt-3.5-turbo-1106', temperature=1.2)

# 設定に反映
Settings.llm = llm
Settings.node_parser = node_parser

# Indexの構築
index = VectorStoreIndex.from_documents(documents)

In [7]:
# 参考：言語モデルのエンコーディングを調べる
import tiktoken
print(tiktoken.encoding_for_model('gpt-3.5-turbo-1106').encode)

<bound method Encoding.encode of <Encoding 'cl100k_base'>>


# テンプレートの作成

In [8]:
# テキストQAプロンプトの作成
from llama_index.core.llms import ChatMessage, MessageRole
from llama_index.core import ChatPromptTemplate

# システムプロンプト
system_prompt = ChatMessage(
    content=("""
        あなたは世界中で信頼されているQAシステムです。
        事前知識ではなく、常に提供されたコンテキスト情報を使用してクエリに回答してください。
        従うべきいくつかのルール:
        1. 回答内で指定されたコンテキストを直接参照しないでください。
        2. 「コンテキストに基づいて、...」や「コンテキスト情報は...」、またはそれに類するような記述は避けてください。"""),
    role=MessageRole.SYSTEM,
)

# メッセージテンプレート
message_templates = [
    system_prompt,
    ChatMessage(
        content=("""
            コンテキスト情報は以下のとおりです。
            ---------------------
            {context_str}
            ---------------------
            事前知識ではなくコンテキスト情報を考慮して、クエリに答えます。
            Query: {query_str}
            Answer: """),
        role=MessageRole.USER,
    ),
]

# テキストQAプロンプト
text_qa_template = ChatPromptTemplate(message_templates=message_templates)

In [9]:
# リファインテンプレートの作成
refine_message_templates = [
    ChatMessage(
        content=("""
            あなたは、既存の回答を改良する際に2つのモードで厳密に動作するQAシステムのエキスパートです。
            1. 新しいコンテキストを使用して元の回答を**書き直す**。
            2. 新しいコンテキストが役に立たない場合は、元の回答を**繰り返す**。
            回答内で元の回答やコンテキストを直接参照しないでください。
            疑問がある場合は、元の答えを繰り返してください。
            New Context: {context_msg}
            Query: {query_str}
            Original Answer: {existing_answer}
            New Answer: """),
        role=MessageRole.USER,
    )
]

# リファインテンプレート
refine_template = ChatPromptTemplate(message_templates=refine_message_templates)

# Query Engineの作成

In [10]:
# Query Engineの作成
query_engine = index.as_query_engine(
    similarity_top_k=3,
    streaming=True,
    text_qa_template=text_qa_template,
    refine_template=refine_template,
)

# ユーザーからの質問に回答

In [11]:
response = query_engine.query("お土産の購入状況を教えて")
# 言語モデルからの回答を表示
response.print_response_stream()

訪日外国人消費動向調査によると、お土産の購入状況は次のようになっています。費目別の購入率では、「菓子類」が最も高い割合で購入されており、「その他食料品・飲料・たばこ」、「衣類」の順で高いことが示されています。さらに、国籍・地域別では、韓国、台湾、香港、中国からの訪日外国人が「菓子類」を特に多く購入しています。また、費目別の購入者単価では、「宝石・貴金属」が最も高いことが分かります。

消費税免税手続きの実施状況では、全体の40.3％の訪日外国人が消費税免税手続きを実施しており、台湾と香港からの訪日外国人ではその割合が60％を超えています。費目別にみると、「衣類」が最も高く免税購入されており、次いで「菓子類」、「医薬品」、「靴・かばん・革製品」の割合が高いことがわかります。

したがって、お土産の購入状況は費目別によって異なり、免税手続きの実施状況も費目別で変化があります。

In [12]:
# 出典を表示
print(response.get_formatted_sources())

> Source (Node id: 0b5e168e-704b-422e-a4f4-750b25db6600): 訪日外国人消費動向調査  
 
20 ３．土産品の購入実態  
 
（１） 費目別購入率 
 費目別の 購入率（その費目を購入した人の
割合）は「菓子類」 （ 70.4％）、「その他食
料品・...

> Source (Node id: 10ffe99b-8cfa-488d-a9d8-8174f9d2ace9): 訪日外国人消費動向調査 
 
38 
 （３）買物場所 
百貨店・デパート 
原則として百貨店協会加盟の店舗 
家電量販店 
PCやカメラ、電気製品を専門に販売する店舗 
ファッション専門店 
...

> Source (Node id: a8ad2164-4db7-4ea3-a15e-bd1d4a803ac2): 21 （３）消費税免税手続き の実施状況  
 今回の日本滞在中に 消費税免税手続き
を実施した人の割合 は全体の40.3％で
ある（図表 3-3）。 
 国籍･地域別に みると、 台湾と香...


# 詳細なログの表示

In [13]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [14]:
response = query_engine.query("お土産の購入状況を教えて")
# 言語モデルからの回答を表示
print(response)

DEBUG:openai._base_client:Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x30a69a0c0>, 'json_data': {'input': ['お土産の購入状況を教えて'], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
Request options: {'method': 'post', 'url': '/embeddings', 'files': None, 'post_parser': <function Embeddings.create.<locals>.parser at 0x30a69a0c0>, 'json_data': {'input': ['お土産の購入状況を教えて'], 'model': 'text-embedding-ada-002', 'encoding_format': 'base64'}}
DEBUG:httpcore.connection:close.started
close.started
DEBUG:httpcore.connection:close.complete
close.complete
DEBUG:httpcore.connection:connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
connect_tcp.started host='api.openai.com' port=443 local_address=None timeout=60.0 socket_options=None
DEBUG:httpcore.connection:connect_tcp.complete return_value=<httpcore._backends.sync.SyncStream object at 0x1741a3dd0>
con

In [15]:
# もとに戻す（ログを出力しない）
logging.basicConfig(stream=sys.stdout, level=logging.WARNING)
# ログハンドラー出力を除去
while len(logging.getLogger().handlers) > 0:
  logging.getLogger().removeHandler(logging.getLogger().handlers[0])

# ストレージに保存

In [16]:
# ストレージに保存
index.storage_context.persist("./storage02")

In [17]:
from llama_index.core import StorageContext, load_index_from_storage

# ストレージコンテキストの作成
storage_context = StorageContext.from_defaults(persist_dir="./storage02")
# Indexのロード
index = load_index_from_storage(storage_context)

In [18]:
# Query Engineの作成
query_engine2 = index.as_query_engine(
    similarity_top_k=3,
    streaming=True,
    text_qa_template=text_qa_template,
    refine_template=refine_template,
)

In [19]:
response = query_engine2.query("お土産の購入状況を教えて")
# 言語モデルからの回答を表示
print(response)

お土産の購入状況について、訪日外国人消費動向調査によれば、菓子類が最も高い購入率であり（70.4％）、次いでその他食料品・飲料・たばこ（43.4％）、衣類（33.8％）の順で高いことが分かります。また、国籍・地域別では韓国、台湾、香港、中国が菓子類を最も購入しており、台湾と香港の宝石・貴金属の購入者単価が他の国籍・地域に比べて高いことも注目されます。
